In [ ]:
#!pip install --quiet textacy
!pip install --quiet wikipedia-api
!pip install adjustText

     |████████████████████████████████| 200 kB 12.9 MB/s 
     |████████████████████████████████| 2.0 MB 67.0 MB/s 
     |████████████████████████████████| 132 kB 62.6 MB/s 
     |████████████████████████████████| 1.6 MB 62.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7097 sha256=89ab6815727e77dde562378b4ceddac35d623e16673acf4082546c66fb6c6fe8
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText


In [ ]:
!git clone https://github.com/dqxiu/ParaSCI.git

In [ ]:
!unzip squad_folds_eval.zip
!unzip hotpotqa_folds_eval.zip

In [ ]:
result = !wget --save-cookies cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p'
code = result[-1]
arg =' --load-cookies cookies.txt "https://docs.google.com/uc?export=download&confirm=%s&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O GoogleNews-vectors-negative300.bin.gz' % code
!wget $arg

In [ ]:
import wikipediaapi
import string
from nltk.tokenize import word_tokenize
import nltk
import textacy
from gensim import models
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
import string
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
#from textacy.similarity.edits import levenshtein
import os
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def get_paraphrase_dataset():

  data = {"text":[], "paraphrased_text":[]}

  paths = ["ParaSCI/Data/ParaSCI-ACL", "ParaSCI/Data/ParaSCI-arXiv"]


  for path in paths:
    for directory in os.listdir(path):
      if directory in ["train", "val", "test"]:
        source = open(path + "/" + directory + "/" + directory + ".src").read().split("\n")
        target = open(path + "/" + directory + "/" + directory + ".tgt").read().split("\n")
        for line in source:
          data["text"].append(line)
        for line in target:
          data["paraphrased_text"].append(line)

  df = pd.DataFrame(data)
  #df.dropna(inplace=True)
  df["is_paraphrase"] = [1] * len(df)
  
  df_false = df.copy()
  data_to_shuffle = df_false["paraphrased_text"]
  
  shuffled_data = data_to_shuffle.sample(frac=1).reset_index(drop=True)
 
  df_false["paraphrased_text"] = list(shuffled_data)
  
  df_false["is_paraphrase"] = [0] * len(df_false)
 
  df = df.sample(500)
  df_false = df_false.sample(500)
  df = df.append(df_false)
  return df


In [ ]:
def _convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

def _doc_to_synsets(doc : string):
    """
    Returns a list of synsets in document.
    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.
    Args:
    doc: string to be converted
    Returns:
    list of synsets
    Example:
    doc_to_synsets('Fish are nvqjp friends.')
    Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """

    doc_tokenized = nltk.word_tokenize(doc)
    doc_tokenized_tagged = nltk.pos_tag(doc_tokenized)
    tags = [x[1] for x in doc_tokenized_tagged]
    new_tags =[_convert_tag(tag) for tag in tags]
    new_doc_tokenized_tagged = list(zip(doc_tokenized, new_tags))
    synsets = [wn.synsets(x[0], x[1])[0] for x in new_doc_tokenized_tagged if len(wn.synsets(x[0], x[1])) > 0]
    return synsets

def _similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2
    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.
    Args:
    s1, s2: list of synsets from doc_to_synsets
    Returns:
    normalized similarity score of s1 onto s2
    Example:
    synsets1 = doc_to_synsets('I like cats')
    synsets2 = doc_to_synsets('I like dogs')
    similarity_score(synsets1, synsets2)
    Out: 0.73333333333333339
    """
    scores = list()
    for x in s1:
        scores.append([x.path_similarity(y) for y in s2 if x.path_similarity(y) != None])
    no_empty_list_scores = [x for x in scores if x !=[]]
    best_scores = [max(x) for x in no_empty_list_scores]
    if len(best_scores) == 0:
        return 0
    normalized_score = sum(best_scores)/len(best_scores)
    return normalized_score

def document_path_similarity(doc1 : string, doc2 : string):
    """Finds the symmetrical similarity between doc1 and doc2"""
    synsets1 = _doc_to_synsets(doc1)
    synsets2 = _doc_to_synsets(doc2)
    return (_similarity_score(synsets1, synsets2) + _similarity_score(synsets2, synsets1)) / 2



def document_vector(doc : string):
  
    doc = word_tokenize(doc)

    words = []

    for word in doc:
      if word.isnumeric():
        for digit in word:
          words.append(digit)
      if list(model.vocab).count(word) == 1:
        words.append(word)
    
    if len(words) > 0:
      return np.mean(model[words], axis=0)

    return []

In [ ]:
def predict_is_paraphrase_dps(row, threshold):

  text1 = str(row["text"])
  text2 = str(row["paraphrased_text"])
  if document_path_similarity(text1,  text2) > threshold:
    return 1
  return 0

def predict_is_paraphrase_word2vec(row, threshold):
  text1 = row["text"]
  text2 = row["paraphrased_text"]
  doc_vec1 = document_vector(str(text1))
  doc_vec2 = document_vector(str(text2))

  if len(doc_vec1) > 0 and len(doc_vec2) > 0:
    similarity = cosine_similarity([doc_vec1], [doc_vec2])[0][0]
    if similarity > threshold:
      return 1
  
  return 0

# def predict_is_paraphrase_levenshtein(row, threshold):
#   text1 = str(row["text"])
#   text2 = str(row["paraphrased_text"])
#   similarity = levenshtein(text1, text2)
#   if similarity > threshold:
#     return 1
#   return 0

In [ ]:
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
df = get_paraphrase_dataset()

In [ ]:
dps_accuracy_scores = []
dps_precision_scores = []
dps_recall_scores = []
dps_f1_scores = []

for i in range(1, 10):
  threshold = i / 10
  df["dps_prediction_" + str(threshold)] = df.apply(lambda x: predict_is_paraphrase_dps(x, threshold), axis = 1)
  dps_accuracy_scores.append(accuracy_score(df["is_paraphrase"], df["dps_prediction_" + str(threshold)]))
  dps_precision_scores.append(precision_score(df["is_paraphrase"], df["dps_prediction_" + str(threshold)]))
  dps_recall_scores.append(recall_score(df["is_paraphrase"], df["dps_prediction_" + str(threshold)]))
  dps_f1_scores.append(f1_score(df["is_paraphrase"], df["dps_prediction_" + str(threshold)]))
  print("Threshold: ", str(threshold))
  print("Accuracy Score: ". str(dps_accuracy_scores[-1]))
  print("Precision Score: ". str(dps_precision_scores[-1]))
  print("Recall Score: ". str(dps_recall_scores[-1]))
  print("F1 Score: ". str(dps_f1_scores[-1]))

In [ ]:
# levenshtein_accuracy_scores = []
# levenshtein_precision_scores = []
# levenshtein_recall_scores = []
# levenshtein_f1_scores = []

# for i in range(1, 10):
#   threshold = i / 10
#   df["levenshtein_prediction_" + str(threshold)] = df.apply(lambda x: predict_is_paraphrase_levenshtein(x, threshold), axis = 1)
#   levenshtein_accuracy_scores.append(accuracy_score(df["is_paraphrase"], df["levenshtein_prediction_" + str(threshold)]))
#   levenshtein_precision_scores.append(precision_score(df["is_paraphrase"], df["levenshtein_prediction_" + str(threshold)]))
#   levenshtein_recall_scores.append(recall_score(df["is_paraphrase"], df["levenshtein_prediction_" + str(threshold)]))
#   levenshtein_f1_scores.append(f1_score(df["is_paraphrase"], df["levenshtein_prediction_" + str(threshold)]))
#   print("Threshold: ", str(threshold))
#   print("Accuracy Score: ". str(levenshtein_accuracy_scores[-1]))
#   print("Precision Score: ". str(levenshtein_precision_scores[-1]))
#   print("Recall Score: ". str(levenshtein_recall_scores[-1]))
#   print("F1 Score: ". str(levenshtein_f1_scores[-1]))



In [ ]:
word2vec_accuracy_scores = []
word2vec_precision_scores = []
word2vec_recall_scores = []
word2vec_f1_scores = []

for i in range(1, 10):
  threshold = i / 10
  df["word2vec_prediction_" + str(threshold)] = df.apply(lambda x: predict_is_paraphrase_word2vec(x, threshold), axis = 1)
  word2vec_accuracy_scores.append(accuracy_score(df["is_paraphrase"], df["word2vec_prediction_" + str(threshold)]))
  word2vec_precision_scores.append(precision_score(df["is_paraphrase"], df["word2vec_prediction_" + str(threshold)]))
  word2vec_recall_scores.append(recall_score(df["is_paraphrase"], df["word2vec_prediction_" + str(threshold)]))
  word2vec_f1_scores.append(f1_score(df["is_paraphrase"], df["word2vec_prediction_" + str(threshold)]))
  print("Threshold: ", str(threshold))
  print("Accuracy Score: ". str(word2vec_accuracy_scores[-1]))
  print("Precision Score: ". str(word2vec_precision_scores[-1]))
  print("Recall Score: ". str(word2vec_recall_scores[-1]))
  print("F1 Score: ". str(word2vec_f1_scores[-1]))